# Wan2GP CPU Prefetch to Volume (Wan2.2 -> ckpts)

Use this on a cheap CPU notebook first to download heavy Wan2.2 files into a persistent volume.

All heavy model files are written under `ckpts` so Wan2GP can load them directly later.


## 1) Configure predictable volume + ckpts paths


In [ ]:
import os
from pathlib import Path

default_volume_root = Path('/models') if Path('/models').exists() else Path('/workspace/wan-volume')
WAN_VOLUME_ROOT = Path(os.environ.get('WAN_VOLUME_ROOT', str(default_volume_root))).resolve()
WAN_CKPTS_CACHE = Path(os.environ.get('WAN_CKPTS_CACHE', str(WAN_VOLUME_ROOT / 'ckpts'))).resolve()
WAN_PREFETCH_SUBDIR = os.environ.get('WAN_PREFETCH_SUBDIR', 'wan22').strip() or 'wan22'
WAN_PREFETCH_PATH = (WAN_CKPTS_CACHE / WAN_PREFETCH_SUBDIR).resolve()

WAN_CKPTS_CACHE.mkdir(parents=True, exist_ok=True)
WAN_PREFETCH_PATH.mkdir(parents=True, exist_ok=True)

print('WAN_VOLUME_ROOT =', WAN_VOLUME_ROOT)
print('WAN_CKPTS_CACHE =', WAN_CKPTS_CACHE)
print('WAN_PREFETCH_PATH =', WAN_PREFETCH_PATH)
print('Downloads in this notebook target WAN_CKPTS_CACHE (ckpts).')


## 2) Install fast downloader dependencies


In [ ]:
import os
import subprocess
import sys
import time

env = os.environ.copy()
env.setdefault('PIP_DISABLE_PIP_VERSION_CHECK', '1')
env.setdefault('PIP_NO_INPUT', '1')
env.setdefault('PIP_PROGRESS_BAR', 'off')
env.setdefault('PIP_DEFAULT_TIMEOUT', '100')
env.setdefault('HF_HUB_ENABLE_HF_TRANSFER', '1')
env.setdefault('HF_HUB_DISABLE_TELEMETRY', '1')
env.setdefault('HF_HUB_DISABLE_PROGRESS_BARS', '1')

setup_dir = WAN_VOLUME_ROOT / '.setup-markers'
setup_dir.mkdir(parents=True, exist_ok=True)
marker = setup_dir / 'prefetch_hfhub.done'

def run_quiet(cmd, label):
    start = time.time()
    print(f'[start] {label}')
    proc = subprocess.run(cmd, env=env, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if proc.returncode != 0:
        tail = '\n'.join(proc.stdout.splitlines()[-120:])
        raise RuntimeError(f'{label} failed (exit {proc.returncode})\n---- output tail ----\n{tail}')
    print(f'[ok] {label} ({time.time() - start:.1f}s)')

if marker.exists():
    print('[skip] downloader dependencies already installed')
else:
    run_quiet([sys.executable, '-m', 'pip', 'install', '--quiet', '--upgrade', 'pip', 'setuptools', 'wheel'], 'pip bootstrap')
    run_quiet([sys.executable, '-m', 'pip', 'install', '--quiet', '--upgrade', 'huggingface_hub[hf_transfer]>=0.24.0'], 'install huggingface_hub[hf_transfer]')
    marker.write_text('ok\n', encoding='utf-8')
    print(f'[ok] wrote marker: {marker.name}')

print('Downloader setup complete.')


## 3) Optional Hugging Face login


In [ ]:
import os
from huggingface_hub import login

token = os.environ.get('HF_TOKEN', '').strip() or os.environ.get('HUGGINGFACE_HUB_TOKEN', '').strip()
if token:
    login(token=token, add_to_git_credential=False)
    print('Hugging Face token detected and login completed.')
else:
    print('No HF token provided; using anonymous downloads (can still work for public models).')


## 4) Download Wan2.2 I2V A14B BF16 files to ckpts


In [ ]:
import os
import time
from pathlib import Path
from huggingface_hub import hf_hub_download

os.environ.setdefault('HF_HUB_ENABLE_HF_TRANSFER', '1')
os.environ.setdefault('HF_HUB_DISABLE_TELEMETRY', '1')
os.environ.setdefault('HF_HUB_DISABLE_PROGRESS_BARS', '1')

download_jobs = [
    {
        'label': 'Wan2.2 I2V 14B high BF16',
        'repo_id': 'DeepBeepMeep/Wan2.2',
        'filename': 'wan2.2_image2video_14B_high_mbf16.safetensors',
        'target': WAN_CKPTS_CACHE / 'wan2.2_image2video_14B_high_mbf16.safetensors',
    },
    {
        'label': 'Wan2.2 I2V 14B low BF16',
        'repo_id': 'DeepBeepMeep/Wan2.2',
        'filename': 'wan2.2_image2video_14B_low_mbf16.safetensors',
        'target': WAN_CKPTS_CACHE / 'wan2.2_image2video_14B_low_mbf16.safetensors',
    },
    {
        'label': 'UMT5 XXL encoder BF16',
        'repo_id': 'DeepBeepMeep/Wan2.1',
        'subfolder': 'umt5-xxl',
        'filename': 'models_t5_umt5-xxl-enc-bf16.safetensors',
        'target': WAN_CKPTS_CACHE / 'umt5-xxl' / 'models_t5_umt5-xxl-enc-bf16.safetensors',
    },
]

start_all = time.time()
for job in download_jobs:
    target = Path(job['target'])
    target.parent.mkdir(parents=True, exist_ok=True)
    if target.exists():
        size_gb = target.stat().st_size / (1024 ** 3)
        print(f"[skip] {job['label']} already exists ({size_gb:.2f} GB)")
        continue

    kwargs = {}
    if job.get('subfolder'):
        kwargs['subfolder'] = job['subfolder']

    start = time.time()
    downloaded = Path(
        hf_hub_download(
            repo_id=job['repo_id'],
            filename=job['filename'],
            local_dir=str(WAN_CKPTS_CACHE),
            **kwargs,
        )
    ).resolve()
    expected = target.resolve()
    if downloaded != expected:
        raise RuntimeError(f"Unexpected local path for {job['label']}: {downloaded} (expected {expected})")

    size_gb = expected.stat().st_size / (1024 ** 3)
    print(f"[ok] {job['label']} ({size_gb:.2f} GB, {time.time() - start:.1f}s)")

marker_file = WAN_CKPTS_CACHE / '.wan22_bf16_prefetch.done'
marker_file.write_text('ok\n', encoding='utf-8')
print(f'[ok] wrote marker: {marker_file}')
print(f'All downloads complete in {time.time() - start_all:.1f}s')


## 5) Validate ckpts contents and print launch env


In [ ]:
required = [
    WAN_CKPTS_CACHE / 'wan2.2_image2video_14B_high_mbf16.safetensors',
    WAN_CKPTS_CACHE / 'wan2.2_image2video_14B_low_mbf16.safetensors',
    WAN_CKPTS_CACHE / 'umt5-xxl' / 'models_t5_umt5-xxl-enc-bf16.safetensors',
]

missing = [str(p) for p in required if not p.exists()]
if missing:
    raise RuntimeError('Missing files after prefetch:\n' + '\n'.join(missing))

print('All required Wan2.2 BF16 files are present in ckpts:')
for path in required:
    size_gb = path.stat().st_size / (1024 ** 3)
    print(f' - {path} ({size_gb:.2f} GB)')

print('\nUse these env vars in cloud_a100_native_webui.ipynb:')
print(f'WAN_CKPTS_CACHE={WAN_CKPTS_CACHE}')
print(f'WAN_PREFETCH_SUBDIR={WAN_PREFETCH_SUBDIR}')
